In [9]:
import cv2 as cv
import numpy as np
import keyboard
import time

import Controlling_thymio
import global_planning
import vision
import loc_avoid
from ekf import ExtendedKalmanFilter
from ekf import apply_kalman

In [ ]:
# Global constants here

IMAGE_WIDTH = 640 # width of the video feed

IMAGE_HEIGHT = 480 # height of the video feed

CAMERA_FPS = 30 # fps of the video feed

KEYBOARD_INPUT = "enter" # keyboard input to start

CAMERA_REFRESH_TIME = 30 # number of frame thrown away to allow the camera to focus in the meantime

SKIPPED_FRAME = 3 # get one frame every X

In [11]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
import Controlling_thymio

node = await client.wait_for_node()

await node.lock()
aw(node.lock())

await node.wait_for_variables({"acc"})

In [ ]:
# 0 is laptop webcam, 1 is USB camera (!COMPUTER DEPENDENT!)
cap = cv.VideoCapture(0)

init = False # boolean set to True on keyboard input to start initialisation
is_init = False # boolean set to True once initialisation has succeeded
black_frame_remaining = 0 # numbers of frames to throw away
being_kidnapped = False # boolean set to True while the robot is being kidnapped
error_linear = [] # array of all linear errors
error_angle = [] # array of all angular errors
initial_turn = True # boolean set to True while the first orientation is not done
frame_number = 0 # to get one frame every X


# variable initialized at False irrelevant of wether it actually is
robot_not_found = False # boolean set to True if robot not found for one frame
camera_hidden = False # boolean set to True if robot not found for multiple frames

# create a robot instance
robot_instance = global_planning.Robot()

# if unable to connect to the camera
if not (cap.isOpened()):
    print("Could not open video device")
else:
    while(True):
        # get the input frame by frame (shape (480,640,3))
        ret, frame = cap.read() 
        cap.set(cv.CAP_PROP_FPS, CAMERA_FPS) 
        frame = cv.resize(frame, (IMAGE_WIDTH,IMAGE_HEIGHT), interpolation=cv.INTER_CUBIC) 

        # wait until keyboard input to initialize vision
        if keyboard.is_pressed(KEYBOARD_INPUT):
            init = True

        # reinitialize the robot after a kidnapping
        if being_kidnapped: 
            if not Controlling_thymio.kidnapping(node):
                
                # to give time to the kidnapper to get out of the frame
                time.sleep(3)
                
                # reinitialize the starting position of the robot
                fop = vision.get_fop(frame, original_coordinates, new_coordinates)
                try:
                    start_x, start_y, robot_angle, width = vision.get_robot_position(fop)
                except:
                    break

                # reinitialize the global planning
                converted_obstacles = [[tuple(arr) for arr in sublist] for sublist in obstacles]
                converted_obstacles = global_planning.obstacle_dictionnary(converted_obstacles)
                goal = (objective_x, objective_y)
                robot_instance.update_coordinates(start_x, start_y, robot_angle, width)
                obstacles_named = global_planning.naming_points(converted_obstacles, robot_instance, goal)
                adg_list = global_planning.creating_adjacency_dictionnary(converted_obstacles, robot_instance, goal)
                path = global_planning.finding_path(adg_list, obstacles_named)

                black_frame_remaining = 0
                camera_hidden = False
                initial_turn = True
                is_init = True
                being_kidnapped = False
        
        # initialize 
        if init == True:

            try:
                # vision: start, obstacles and goal coordinates
                original_coordinates, new_coordinates = vision.get_fop_coordinates(frame)
                fop = vision.get_fop(frame, original_coordinates, new_coordinates)
                start_x, start_y, robot_angle, width = vision.get_robot_position(fop)
                obstacles = vision.get_obstacles(fop, width)
                objective_x, objective_y = vision.get_objective(fop)

                # global planning
                converted_obstacles = [[tuple(arr) for arr in sublist] for sublist in obstacles]
                converted_obstacles = global_planning.obstacle_dictionnary(converted_obstacles)
                goal = (objective_x,objective_y)
                robot_instance.update_coordinates(start_x, start_y, robot_angle,width)
                obstacles_named = global_planning.naming_points(converted_obstacles,robot_instance,goal)
                adg_list = global_planning.creating_adjacency_dictionnary(converted_obstacles,robot_instance,goal)
                path = global_planning.finding_path(adg_list,obstacles_named)
                
                # extended kalman filter
                ekf = ExtendedKalmanFilter([center_x,center_y,robot_angle,0,0])
                ekf.set_time_t(time.time())

                black_frame_remaining = 0
                camera_hidden = False
                being_kidnapped = False
                is_init = True
            except Exception as e:
                print("Initialisation failed: " + str(e))

            init = False

        if is_init:  
        ################################## MOTION CONTROL HERE ################################## 
            
            # get out of the loop if the robot is being kidnapped
            if Controlling_thymio.kidnapping(node):
                being_kidnapped = True
                is_init = False
                continue

            fop = vision.get_fop(frame, original_coordinates, new_coordinates)

            # to use one frame every X
            if frame_number%SKIPPED_FRAME == 0:

                try:
                    center_x, center_y, robot_angle, width = vision.get_robot_position(fop)
                    robot_instance.update_coordinates(robot_instance, center_x, center_y, robot_angle, width)
                    robot_not_found = False
                except:
                    center_x = 0
                    center_y = 0
                    robot_angle = 0
                    if robot_not_found:
                        camera_hidden = True
                    robot_not_found = True

            frame_number += 1
            
            # ekf update
            position = [center_x, center_y, robot_angle[0]]
            speed = [int(left_wheel_speed),int(right_wheel_speed)]
            
            try:
                x, _ = apply_kalman(ekf, not camera_hidden, position, speed)
                new_position = x[0:3]  # Extract the filtered position
            except ValueError as e:
                print(f"Error during Kalman filter update: {e}")

                new_position = [center_x, center_y, robot_angle]

            # camera was covered, now uncovered
            if center_x != 0 and center_y != 0 and camera_hidden:
                camera_hidden = False
                black_frame_remaining = CAMERA_REFRESH_TIME # to give it time to gain focus

                # position updated using Kalman
                robot_instance.update_coordinates(new_position[0], new_position[1], new_position[2], width)

            # camera was uncovered, now covered
            elif center_x == 0 and center_y == 0:
                # position updated using Kalman
                robot_instance.update_coordinates(new_position[0], new_position[1], new_position[2], width)
            
            # camera was recently uncovered, set some frames to black to allow it time to gain focus
            elif black_frame_remaining > 0:
                # set the first frames after uncovering to black to allow time to focus -> navigation using extended Kalman filter
                fop = cv.cvtColor(fop, cv.COLOR_BGR2GRAY)
                _, fop = cv.threshold(fop,255,255,cv.THRESH_BINARY)
                black_frame_remaining -= 1

                # position updated using Kalman
                robot_instance.update_coordinates(new_position[0], new_position[1], new_position[2], width)
            
            fop = cv.circle(fop, (robot_instance.center_x,robot_instance.center_y), radius=10, color=(0, 255, 0), thickness=-1)

            start_point = np.array([obstacles_named[path[0]][0],obstacles_named[path[0]][1]])
            end_point = np.array([obstacles_named[path[1]][0],obstacles_named[path[1]][1]])
            
            # traget point after the actual end point
            try:
                next_point = np.array([obstacles_named[path[2]][0],obstacles_named[path[2]][1]])
            except:
                next_point = [0, 0]

            # proximity avoidance
            avoiding_obstacle, left_wheel_speed, right_wheel_speed = loc_avoid.local_avoidance(node,robot_instance,aw)

            # update of the wheel speed
            if not avoiding_obstacle:
                left_wheel_speed, right_wheel_speed = Controlling_thymio.compute_wheel_speed(initial_turn, start_point, end_point, next_point, robot_instance, aw, node)

            # draw everything
            #if not camera_hidden and black_frame_remaining <= 0:
            cv.circle(fop, (objective_x, objective_y), radius=10, color=(255, 0, 0), thickness=-1)
            cv.circle(fop, (start_x, start_y), radius=10, color=(0, 0, 255), thickness=-1)
            for obstacle in obstacles:
                for vertice in obstacle:
                    cv.circle(fop, (vertice[0],vertice[1]), radius=10, color=(0, 255, 0), thickness=-1)
            for i in range(len(path[:-1])):
                cv.line(fop,(obstacles_named[path[i]][0],obstacles_named[path[i]][1]),(obstacles_named[path[i+1]][0],obstacles_named[path[i+1]][1]),(255,0,0),5)
    
        #########################################################################################
        else:
            # to output the raw feed
            fop = frame

        cv.imshow('Raw',frame)
        cv.imshow('Processed',fop)

        await client.sleep(0.001)
        
        # Close all windows and get out of the loop if ESC is pressed
        if cv.waitKey(1) == 27:
            cv.destroyAllWindows()
            break

['R', 'P3', 'G']
robot position : 23 324 [0.74837805]


C:\Users\mehdi\AppData\Local\Temp\ipykernel_31472\4136444558.py:152: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  Controlling_thymio.set_speed(int(left_wheel_speed),int(right_wheel_speed),aw,node)


robot position : 27 318 [0.78539816]
robot position : 29 313 [0.]
robot position : 29 309 [0.95054684]
robot position : 33 303 [-0.88506682]
Kalmman position : [25, 242, 0]
Kalmman position : [31, 286, 0]
robot position : 34 302 [0.41822433]
Kalmman position : [34, 296, 0]
Kalmman position : [34, 300, 0]
Kalmman position : [35, 302, 0]
robot position : 36 301 [0.46364761]
Kalmman position : [37, 301, 0]
Kalmman position : [36, 301, 0]
Kalmman position : [36, 301, 0]
robot position : 37 301 [-0.26625205]
Kalmman position : [37, 301, 0]
Kalmman position : [38, 300, 0]
Kalmman position : [38, 300, 0]
robot position : 41 301 [1.62955215]
Kalmman position : [41, 300, 1]
Kalmman position : [41, 301, 1]
Kalmman position : [41, 302, 1]
robot position : 40 300 [1.57079633]
Kalmman position : [40, 302, 1]
Kalmman position : [40, 301, 1]
Kalmman position : [40, 301, 1]
robot position : 41 300 [-0.11065722]
Kalmman position : [40, 301, 0]
Kalmman position : [42, 300, 0]
Kalmman position : [41, 300

CancelledError: 